## Analyzing Dialysis Facilities in the United States  
This project will analyze dialysis facilities in the United States and attempt to answer the following question:
 - What factors lead to better care in a dialysis facility?

https://github.com/carl-schick-ds/meteorite-landings

https://catalog.data.gov/dataset/medicare-dialysis-facilities